In [ ]:
import pandas as pd

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
seir_data = pd.read_csv(
    "./csvs/model_4_2.csv",
    header=42,
    index_col=0,
    nrows=401,
    usecols=range(5),
)

In [ ]:
def build_seir_model(
    config: dict,
) -> CompartmentalModel:
    
    # Model characteristics
    compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
    model = CompartmentalModel(
        times=(0, config["end_time"]),
        compartments=compartments,
        infectious_compartments=["Infectious"],
        timestep=config["t_step"],
    )
    n_immune_0 = config["prop_immune_0"] * config["tot_popn"]
    model.set_initial_population(
        distribution={
            "Susceptible": config["tot_popn"] - config["infous_0"] - n_immune_0, 
            "Infectious": config["infous_0"],
            "Immune": n_immune_0,
        }
    )
    
    # Transitions
    ave_infous = Parameter("ave_infous")
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("r0") / ave_infous,
        source="Susceptible",
        dest="Pre-infectious"
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=1. / Parameter("ave_preinfous"),
        source="Pre-infectious", 
        dest="Infectious"
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / ave_infous,
        source="Infectious", 
        dest="Immune"
    )
    
    # Output
    model.request_output_for_flow(
        name="incidence", 
        flow_name="progression"
    )
    model.request_function_output(
        name="incidence_per_tstep",
        func=DerivedOutput("incidence") * config["t_step"],
    )
    return model

In [ ]:
model_config = {
    "tot_popn": 5234.,
    "infous_0": 2.,
    "end_time": 200.,
    "t_step": 0.5,
    "prop_immune_0": 0.3,
}

parameters = {
    "r0": 2.1,
    "ave_preinfous": 2.,
    "ave_infous": 2.,
    "life_expectancy": 70.,
}

In [ ]:
seir_model_part_immune = build_seir_model(model_config)
seir_model_part_immune.run(parameters=parameters, solver="euler")
seir_values = seir_model_part_immune.get_outputs_df()

In [ ]:
assert abs(seir_data - seir_values).max().max() < 1e5